In [1]:
from pyspark.ml.feature import OneHotEncoder,StringIndexer,VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import col, stddev_samp

In [2]:
# read data - 수집
# data processing - EDA
# model create or choice
# training
# predict(classification / regression)
# score
# 배포 - 엔지니어링

In [3]:
df = spark.read.format('csv')\
    .options(header = 'true', inferSchema = 'true')\
    .load('Default.csv').drop('_c0').cache()

In [4]:
df.show(2)

+-------+-------+-----------+-----------+
|default|student|    balance|     income|
+-------+-------+-----------+-----------+
|     No|     No|729.5264952|44361.62507|
|     No|    Yes|817.1804066| 12106.1347|
+-------+-------+-----------+-----------+
only showing top 2 rows



In [5]:
df.select("default").distinct().show()

+-------+
|default|
+-------+
|     No|
|    Yes|
+-------+



In [6]:
# **params # unpacking
# StringIndexer(outputCol=

params = {
    'inputCol':'student',
    'outputCol':'studentIdx'
}
strIdx = StringIndexer(**params)

In [7]:
params = {
    'inputCol':'default',
    'outputCol':'label'
}
label_strIdx = StringIndexer(**params)

In [8]:
# 수치ㅇ 데이터에 대해서
# 의류사이즈 2XL XL L M S --> labeling(4,3,2,1,0) -> onehot(10000, 01000, 00100, 00001) : 이 경우 onehot으로 하ㄴ 안됨
params = {
    'inputCol':'studentIdx',
    'outputCol':'studentClassVec'
}
encode = OneHotEncoder(**params)

In [9]:
stage = [strIdx, encode, label_strIdx]

In [10]:
## 범주형 데이터에 대한 전처리 준비 끝..

In [11]:
## 연속형 데이터 / 수치형 데이터

In [12]:
numCols = ['income','balance']
# scaling
for c in numCols:
    df = df.withColumn(c+'Scaled', col(c) / df.agg(stddev_samp(c)).first()[0])

In [13]:
df.show(2)

+-------+-------+-----------+-----------+------------------+------------------+
|default|student|    balance|     income|      incomeScaled|     balanceScaled|
+-------+-------+-----------+-----------+------------------+------------------+
|     No|     No|729.5264952|44361.62507|3.3262970676634867|1.5081742710178534|
|     No|    Yes|817.1804066| 12106.1347|0.9077350139857981|1.6893841034192338|
+-------+-------+-----------+-----------+------------------+------------------+
only showing top 2 rows



In [14]:
inputs = ['studentClassVec', 'incomeScaled','balanceScaled']

In [15]:
assembler = VectorAssembler(inputCols=inputs,outputCol='features')

In [16]:
stage +=[assembler]
stage

[StringIndexer_8bee5c44a694,
 OneHotEncoder_3f82fffcea98,
 StringIndexer_75fa230fe0fe,
 VectorAssembler_a57a86837349]

In [17]:
# pipe line 
pipeline = Pipeline(stages=stage)
pipelineModel =  pipeline.fit(df)
dataset =  pipelineModel.transform(df)


In [18]:
dataset.show(10)

+-------+-------+-----------+-----------+------------------+------------------+----------+---------------+-----+--------------------+
|default|student|    balance|     income|      incomeScaled|     balanceScaled|studentIdx|studentClassVec|label|            features|
+-------+-------+-----------+-----------+------------------+------------------+----------+---------------+-----+--------------------+
|     No|     No|729.5264952|44361.62507|3.3262970676634867|1.5081742710178534|       0.0|  (1,[0],[1.0])|  0.0|[1.0,3.3262970676...|
|     No|    Yes|817.1804066| 12106.1347|0.9077350139857981|1.6893841034192338|       1.0|      (1,[],[])|  0.0|[0.0,0.9077350139...|
|     No|     No|1073.549164|31767.13895|2.3819447770614217|2.2193837214557224|       0.0|  (1,[0],[1.0])|  0.0|[1.0,2.3819447770...|
|     No|     No|529.2506047|35704.49394|2.6771731943459827|1.0941372934102322|       0.0|  (1,[0],[1.0])|  0.0|[1.0,2.6771731943...|
|     No|     No|785.6558829|38463.49588|2.8840470419162356|1.

In [19]:
# train, test, split -- why?
# 7 : 3
# 7.5 : 2.5
(train,test) = dataset.randomSplit([0.7,0.3],seed=14)

In [20]:
train.count(), test.count(), dataset.count()

(6907, 3093, 10000)

In [21]:
# 학습용과 평가용 데이터 준비 끝~~ 결측치와 이상치, 그리고 피처엔지니어링과 같은 고급 기법은 적용안함)
# 단지 스케일만 맞줘줌

In [24]:
# 적절한 모델 준비
lr = LogisticRegression(labelCol='label',featuresCol="features", maxIter=10)

In [25]:
lrModel = lr.fit(train) #훈련
predictions = lrModel.transform(test) # 예측
predictions.select('default','prediction').show(5)

24/03/26 16:52:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


+-------+----------+
|default|prediction|
+-------+----------+
|     No|       0.0|
|     No|       0.0|
|     No|       0.0|
|     No|       0.0|
|     No|       0.0|
+-------+----------+
only showing top 5 rows



In [26]:
# 평가

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [30]:
evaluator =  BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')
evaluator.evaluate(predictions)  # AUC (Binary Classification Evaluator)

0.9549682684102574

In [ ]:
# 위와 같은 방식의 평가는 일반화 시킬 수 있을까
# 교차 검증을 통해 신뢰성 확보
# 각종 파라메터의 값이 변경됨에 따라 모델의 성능이 달라진다
# 하이퍼 파라미터 튜닝 기법